<a href="https://colab.research.google.com/github/luqmanoxyz/Automatic-Speech-Recognition/blob/main/Transcribing_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate optimum
!pip install -q ipython-autotime
!pip install python-docx

In [2]:
%load_ext autotime

time: 355 µs (started: 2023-11-01 23:48:52 +00:00)


In [3]:
import torch
from transformers import pipeline
from docx import Document
from google.colab import files

time: 20 s (started: 2023-11-01 23:48:53 +00:00)


In [4]:
#Load Audio data file
!wget https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/sam_altman_lex_podcast_367.flac

--2023-11-01 23:49:13--  https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/sam_altman_lex_podcast_367.flac
Resolving huggingface.co (huggingface.co)... 65.8.178.93, 65.8.178.118, 65.8.178.27, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.93|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/96/e4/96e4f69cd112b019dd764318570e47e5fe96de53d8c32a99d745e72d9086e355/b2fd593ce144a8d904cf49a4ed77ed06eb50644a053dddd280c81a3ef94fb60e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sam_altman_lex_podcast_367.flac%3B+filename%3D%22sam_altman_lex_podcast_367.flac%22%3B&response-content-type=audio%2Fx-flac&Expires=1699141753&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTE0MTc1M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85Ni9lNC85NmU0ZjY5Y2QxMTJiMDE5ZGQ3NjQzMTg1NzBlNDdlNWZlOTZkZTUzZDhjMzJhOTlkNzQ1ZTcyZDkwODZlMzU1L2Iy

In [5]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v2",
                torch_dtype=torch.float16,
                device="cuda:0")

pipe.model = pipe.model.to_bettertransformer()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


time: 2min 53s (started: 2023-11-01 23:49:23 +00:00)


In [6]:
outputs = pipe("sam_altman_lex_podcast_367.flac",
               chunk_length_s=30,
               batch_size=24,
               return_timestamps=True)

time: 5min 21s (started: 2023-11-01 23:52:16 +00:00)


In [ ]:
outputs["text"][:200]

In [9]:
'''
from IPython.display import Audio, display

# Load the audio file
audio_file_path = "sam_altman_lex_podcast_367.flac"
audio = Audio(filename=audio_file_path)

# Display the audio player
display(audio)

In [7]:
# Extracting all of the transcription text
text_to_save = outputs["text"]

# Creating a new Document
doc = Document()
doc.add_paragraph(text_to_save)

# Saving the document
doc.save('full_transcription.docx')


time: 111 ms (started: 2023-11-01 23:58:00 +00:00)


In [8]:
# Saving the document
doc.save('sam_altman_lexf_transcribed.docx')

# Downloading the file
files.download('sam_altman_lexf_transcribed.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 54.9 ms (started: 2023-11-01 23:58:03 +00:00)
